In [10]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 28 16:16:39 2017

@author: 麦芽的香气
"""
import pandas as pd
import numpy as np
from sklearn import  preprocessing
import xgboost as xgb
import lightgbm as lgb    
from tqdm import tqdm
path='./data/'
df=pd.read_csv(path+u'训练数据-ccf_first_round_user_shop_behavior.csv')
shop=pd.read_csv(path+u'训练数据-ccf_first_round_shop_info.csv')
test=pd.read_csv(path+u'AB榜测试集-evaluation_public.csv')
df=pd.merge(df,shop[['shop_id','mall_id']],how='left',on='shop_id')
df['time_stamp']=pd.to_datetime(df['time_stamp'])
train=pd.concat([df,test])

In [11]:
mall_list=list(set(list(shop.mall_id)))
result=pd.DataFrame()
for mall in tqdm(mall_list):
    train1=train[train.mall_id==mall].reset_index(drop=True)       
    l=[]
    wifi_dict = {}
    for index,row in train1.iterrows():
        wifi_list = [wifi.split('|') for wifi in row['wifi_infos'].split(';')]
        for i in wifi_list:
            row[i[0]]=int(i[1])
            if i[0] not in wifi_dict:
                wifi_dict[i[0]]=1
            else:
                wifi_dict[i[0]]+=1
        l.append(row)    
    delate_wifi=[]
    for i in wifi_dict:
        if wifi_dict[i]<20: # 加入出现次数少于20的wifi bssid
            delate_wifi.append(i)
    m=[]
    for row in l:
        new={}
        for n in row.keys():
            if n not in delate_wifi:
                new[n]=row[n] # 过滤掉delate_wifi内的wifi bssid
        m.append(new)
    train1=pd.DataFrame(m)
    df_train=train1[train1.shop_id.notnull()]
    df_test=train1[train1.shop_id.isnull()]
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train['shop_id'].values))
    df_train['label'] = lbl.transform(list(df_train['shop_id'].values))    
    num_class=df_train['label'].max()+1    
    params = {
            'objective': 'multi:softmax',
            'eta': 0.1,
            'max_depth': 9,
            'eval_metric': 'merror',
            'seed': 0,
            'missing': -999,
            'num_class':num_class,
            'silent' : 1
            }
    feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','time_stamp','mall_id','wifi_infos']]    
    xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
    xgbtest = xgb.DMatrix(df_test[feature])
    watchlist = [ (xgbtrain,'train'), (xgbtrain, 'test') ]
    num_rounds=60
    model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=15)
    df_test['label']=model.predict(xgbtest)
    df_test['shop_id']=df_test['label'].apply(lambda x:lbl.inverse_transform(int(x)))
    r=df_test[['row_id','shop_id']]
    result=pd.concat([result,r])
    result['row_id']=result['row_id'].astype('int')
    result.to_csv(path+'sub.csv',index=False)

  0%|          | 0/97 [00:00<?, ?it/s]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	train-merror:0.18799	test-merror:0.18799
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.154867	test-merror:0.154867
[2]	train-merror:0.134818	test-merror:0.134818
[3]	train-merror:0.119031	test-merror:0.119031
[4]	train-merror:0.106731	test-merror:0.106731
[5]	train-merror:0.098499	test-merror:0.098499
[6]	train-merror:0.090654	test-merror:0.090654
[7]	train-merror:0.086102	test-merror:0.086102
[8]	train-merror:0.079322	test-merror:0.079322
[9]	train-merror:0.071768	test-merror:0.071768
[10]	train-merror:0.067215	test-merror:0.067215
[11]	train-merror:0.06276	test-merror:0.06276
[12]	train-merror:0.058789	test-merror:0.058789
[13]	train-merror:0.053947	test-merror:0.053947
[14]	train-merror:0.050073	test-merror:0.050073
[15]	train-merror:0.046973	test-merror:0.046973
[16]	train-merror:0.044165	test-merror:0.044165
[17]	train-merror:0.042324	test-merror:0.042324
[18

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  1%|          | 1/97 [05:02<8:04:41, 302.94s/it]

[0]	train-merror:0.044424	test-merror:0.044424
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.038682	test-merror:0.038682
[2]	train-merror:0.03566	test-merror:0.03566
[3]	train-merror:0.032739	test-merror:0.032739
[4]	train-merror:0.030624	test-merror:0.030624
[5]	train-merror:0.028206	test-merror:0.028206
[6]	train-merror:0.026393	test-merror:0.026393
[7]	train-merror:0.023673	test-merror:0.023673
[8]	train-merror:0.021154	test-merror:0.021154
[9]	train-merror:0.01924	test-merror:0.01924
[10]	train-merror:0.017729	test-merror:0.017729
[11]	train-merror:0.016621	test-merror:0.016621
[12]	train-merror:0.016319	test-merror:0.016319
[13]	train-merror:0.015413	test-merror:0.015413
[14]	train-merror:0.01511	test-merror:0.01511
[15]	train-merror:0.014304	test-merror:0.014304
[16]	train-merror:0.013901	test-merror:0.013901
[17]	train-merror:0.013096	test-merror:0.013096
[18]	

  2%|▏         | 2/97 [08:36<7:17:09, 276.10s/it]

KeyboardInterrupt: 